# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

<div> <b>Цель исследования:</b><br>
    1. Построить различные модели классификации коментариев <br>
    2. Добиться значения F1 не ниже 0.75<br><br>
    <b>Ход исследования:</b><br>
    1. Загрузка и пердобработка данных<br>
    2. Построение моделей<br>
    3. Пулучение нужного результата и анализ моделей<br>
    </div>

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


import nltk
import spacy
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer


import re

from catboost import CatBoostClassifier

In [2]:
df = pd.read_csv('toxic_comments.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [3]:
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


Почистим от лишних символов и лематизируем данные.

In [4]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def lemmatize(text):

    word_list = nlp(text)
    lemmatized_output = ' '.join([token.lemma_ for token in word_list])
        
    return lemmatized_output


def clear_text(text):
    c = re.sub(r'[^a-zA-Z \']', ' ', text)
    c = ' '.join(c.split())
    return(c.lower())

In [5]:
df['c_text'] = df['text'].apply(clear_text)

Удалим не содержательные строки

In [6]:
df[df['c_text'] == '']

,text,toxic,c_text
4475,1993\n\n1994\n\n1995\n\n1996\n\n1997\n\n1998\n...,0,
6289,193.61.111.53 15:00,0,
10193,"64.86.141.133""",0,
17280,~ \n\n68.193.147.157,0,
38743,"88.104.31.21""",0,
52336,"14:53,",0,
53679,92.24.199.233|92.24.199.233]],0,
61644,"""\n\n 199.209.144.211 """,0,
119018,"""""""",1,
137400,"== """"""",0,


In [7]:
df = df[df['c_text'] != '']
df.shape

(159282, 3)

In [9]:
tqdm.pandas()

df['lemm_text'] = df['c_text'].progress_apply(lemmatize)

  0%|          | 0/159282 [00:00<?, ?it/s]

B:\programs\Temp\ipykernel_2380\2562809883.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lemm_text'] = df['c_text'].progress_apply(lemmatize)


In [10]:
df = df[['toxic', 'lemm_text']]
df.to_csv('toxic_comments_lemm.csv')

In [8]:
df = pd.read_csv('toxic_comments_lemm.csv')

Поделим получившиеся данные на трейн, тест и валидайию

In [9]:
train_full, test = train_test_split(df, test_size=0.20, random_state=123, stratify=df['toxic'])
train, valid = train_test_split(train_full, test_size=0.20, random_state=123, stratify=train_full['toxic'])

In [10]:
del(nlp, df)

## Обучение

Обучим TF-IDF на обучающей выборке.

In [11]:
#corpus = train['lemm_text'].values.astype('U')

stopwords = set(nltk_stopwords.words('english'))

tf_idf = TfidfVectorizer(stop_words=stopwords).fit(train['lemm_text'])

Обучим несколько моделей.

### Логистическая регрессия

In [12]:
X = tf_idf.transform(train['lemm_text'])
y = train['toxic']
val = tf_idf.transform(valid['lemm_text'])
f_1 = 0.74

In [ ]:
for c in notebook.tqdm(range(3,6)):
    model = LogisticRegression(penalty = 'l1', class_weight = 'balanced', random_state=123, solver='liblinear', C = c, max_iter=200)
    model.fit(X, y)
    f = f1_score(valid['toxic'], model.predict(val))
    if f > f_1:
        model_lin = model
        f_1 = f
        alpha = c
print(f_1, alpha, sep='\n')

  0%|          | 0/3 [00:00<?, ?it/s]

Метрика удавлетворяет поставленной задаче.

### CatBoost

In [12]:
X = tf_idf.transform(train['lemm_text'])
y = train['toxic']

model = CatBoostClassifier(iterations=300, verbose=0)
model.fit(X, y)

f1_score(valid['toxic'], model.predict(tf_idf.transform(valid['lemm_text'])))

0.739881764438381

Метрика на валидации не удовлетворяет поставленной задаче.

### CatBoost внутренний метод

In [14]:
model = CatBoostClassifier(iterations=300, text_features = ['lemm_text'], verbose=0)
model.fit(train.drop(['toxic'],axis=1), train['toxic'])

f1_score(valid['toxic'], model.predict(valid.drop(['toxic'],axis=1)))

Learning rate set to 0.22386
0:	learn: 0.4502146	total: 77.8ms	remaining: 23.3s
100:	learn: 0.1081248	total: 9.17s	remaining: 18.1s
200:	learn: 0.0958787	total: 18.2s	remaining: 8.97s
299:	learn: 0.0886615	total: 27.2s	remaining: 0us


0.7815896015341999

Данная модель показывает лучший результат на валидации. 

## Тест

Протестируем лучшую модель.

In [43]:
model = CatBoostClassifier(iterations=300, text_features = ['lemm_text'], verbose=100)
model.fit(train_full.drop(['toxic'],axis=1), train_full['toxic'])

f1_score(test['toxic'], model.predict(test.drop(['toxic'],axis=1)))

Learning rate set to 0.246246
0:	learn: 0.3949720	total: 480ms	remaining: 2m 23s
100:	learn: 0.1082417	total: 49.5s	remaining: 1m 37s
200:	learn: 0.0976452	total: 1m 37s	remaining: 48.3s
299:	learn: 0.0902273	total: 2m 26s	remaining: 0us


0.7920054200542005

In [21]:
tf_idf = TfidfVectorizer(stop_words=stopwords).fit(train_full['lemm_text'])
X = tf_idf.transform(train_full['lemm_text'])
y = train_full['toxic']
val = tf_idf.transform(test['lemm_text'])
model_lin.fit(X,y)
f1_score(test['toxic'], model_lin.predict(val))

0.76451476197203

Логистическая регрессия также показывает неплохой результат на TF-IDF.

## Выводы

Были даны размеченные данные с коментариями пользователей. Проведена чистка текстов и их лематизация с помощью spacy. Удалены не заначащие коментарии. 

Были обучены 2 модели (LogisticRegression и CatBoostClassifier) на TF-IDF. CatBoost получила необходимую точность на валидации. 

Также была обучена модель CatBoostClassifier с внутренним методом обработки текста. Данная модель показала лучшую точность. На тесте метрика F1 получилась 0.79.